In [1]:
! pip install accelerate peft bitsandbytes pip install git+https://github.com/huggingface/transformers trl py7zr  optimum

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-cv3fk34e
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-cv3fk34e
  Resolved https://github.com/huggingface/transformers to commit cc3e4781854a52cf090ffde28d884a527dab6708
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.5/396.5 kB

In [6]:
!git clone https://github.com/PanQiWei/AutoGPTQ.git

Cloning into 'AutoGPTQ'...
remote: Enumerating objects: 3724, done.
remote: Counting objects: 100% (878/878), done.
remote: Compressing objects: 100% (295/295), done.
remote: Total 3724 (delta 650), reused 707 (delta 580), pack-reused 2846
Receiving objects: 100% (3724/3724), 7.81 MiB | 7.36 MiB/s, done.
Resolving deltas: 100% (2444/2444), done.


In [13]:
cd /content/AutoGPTQ

/content/AutoGPTQ


In [17]:
! git checkout ef442d9

Previous HEAD position was eea67b7 Merge pull request #256 from PanQiWei/rocm_build_bug_fix
HEAD is now at ef442d9 Fix setuptools classifier (#285)


In [18]:
!pip install .


Processing /content/AutoGPTQ
  Preparing metadata (setup.py) ... done
  Created wheel for auto-gptq: filename=auto_gptq-0.4.2+cu1180-cp310-cp310-linux_x86_64.whl size=8342043 sha256=269af95ae9575fbce64a7e2067039ac976293ced7043a010da8e50a893d469e4
  Stored in directory: /tmp/pip-ephem-wheel-cache-37ysv8rn/wheels/8c/41/f4/48ea4848ab4977e74d11a4abbc2c42745c5b1d33f931e8cadf
Successfully built auto-gptq
  Attempting uninstall: auto-gptq
    Found existing installation: auto-gptq 0.4.1+cu1180
    Uninstalling auto-gptq-0.4.1+cu1180:
      Successfully uninstalled auto-gptq-0.4.1+cu1180


In [26]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#Dataset link https://huggingface.co/datasets/Kabatubare/medical

In [27]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os

def finetune_mistral_7b():
    data = load_dataset("Kabatubare/medical", split="train")
    data_df = data.to_pandas()
    data_df["text"] = data_df[["Question", "Answer"]].apply(lambda x: "###Human: Help me with Medical Suggestions: " + x["Question"] + "\n###Assistant: " +x["Answer"], axis=1)
    print(data_df.iloc[0])
    data = Dataset.from_pandas(data_df)
    tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")
    tokenizer.pad_token = tokenizer.eos_token
    quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)
    model = AutoModelForCausalLM.from_pretrained(
                              "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
                              quantization_config=quantization_config_loading,
                              device_map="auto"
                        )
    print(model)
    model.config.use_cache=False
    model.config.pretraining_tp=1
    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)
    peft_config = LoraConfig(
        r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
    )
    model = get_peft_model(model, peft_config)
    training_arguments = TrainingArguments(
        output_dir="mistral7b-finetuned-medical",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        num_train_epochs=1,
        max_steps=250, #250*8=2000 data samples only trained
        fp16=True,
        push_to_hub=True
    )
    trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=512
    )
    trainer.train()
    trainer.push_to_hub()

if __name__ == "__main__":
    finetune_mistral_7b()

Context     You are a medical knowledge assistant trained ...
Question    my 5 1/2-year-old son displays adhd symptoms f...
Answer      adhd and bipolar mood disorder (bmd) can coexi...
text        ###Human: Help me with Medical Suggestions: my...
Name: 0, dtype: object


Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (rotary_emb): MistralRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)


Map:   0%|          | 0/23437 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:214: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the diff

Step,Training Loss
100,2.113300
200,1.999900


In [30]:
! cp -r /content/mistral7b-finetuned-medical /content/drive/MyDrive/

cp: cannot stat '/content/mistral7b-finetuned-medical': No such file or directory


In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("/content/mistral7b-finetuned-medical")

inputs = tokenizer("""
###Human: Help me with Medical Suggestions: I've Symptoms of heavy coughing for last one month also I ve severe migraine,stomachache.What should i do now?
###Assistant: """, return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/mistral7b-finetuned-medical",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=25,
    pad_token_id=tokenizer.eos_token_id
)

OSError: ignored

In [ ]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)